# Tentamen Functioneel Programmeren 2022-2023 kans 1

Dit is het tweede deel (van in totaal twee delen). Beantwoord de vragen die in Remindo staan in dit notebook en lever het daarna in via Remindo. Je mag alle hulpmiddelen bij dit deel gebruiken. Dit deel van het tentamen bestaat uit 63 punten (70% van het totaal van 90 punten). 

### Vraag 1

In [1]:
// 1. Closure: wel
// 2. Functiecompositie: wel
// 3. Partial function application: wel
// 4. Recursie: wel
// 5. Pattern matching: wel
// 6. Pure functie: wel
// 7. Onpure functie: wel
// 8. Tuple: wel
// 9. Discriminated union (enum): wel
// 10. Lambda: wel

// Dit is een soort visitor pattern in een boom in pre-order (een boom is ook wel een soort composite pattern)!

type TreeNode = // enum
    | Empty
    | Leaf of int
    | Left of int * TreeNode
    | Right of int * TreeNode
    | LeftAndRight of int * TreeNode * TreeNode

let rec forEach node visit = // recursie, pure functie
    match node with // pattern matching
        | Empty -> ()
        | Leaf i -> visit i
        | Left (i, n) | Right (i, n) ->  // tuple
            visit i
            forEach n visit
        | LeftAndRight (i, l, r) -> // tuple
            visit i
            forEach l visit
            forEach r visit


let prettyPrint n = // onpure functie
    printfn "Found number: %i" n
    n

let multipleOf d = // pure functie
    fun n -> if n % d = 0 then 1 else 0 // lambda, closure, pure functie

let mutable count = 0

let increaseCounter amount = count <- count + amount // onpure functie, closure

let visitor = (multipleOf 3) >> increaseCounter // compositie, partial application mag ook hier tellen, onpure functie

let tree = LeftAndRight (3, Left (3, Leaf (7)), Left (21, Right (22, Leaf(9)))) // tuple

let traverseTreeWith = forEach tree // partial application

traverseTreeWith visitor

printfn "There are %i values in the tree that are divisible by 3" count

There are 4 values in the tree that are divisible by 3


### Vraag 2

In [30]:
type IPrintable =
    abstract Print: unit -> unit

type Shape = // FP: discriminated union, om inheritance/polymorfisme te vergemakkelijken
    | Circle of float
    | EquilateralTriangle of float
    | Square of float
    | Rectangle of float * float

    member this.Area = // OO: methoden/attributen, om gedrag en state bij elkaar te houden (encapsulation)
        match this with // FP: pattern matching, om meer declaratief te programmeren
        | Circle r -> Math.PI * (r ** 2.0)
        | EquilateralTriangle s -> s * s * sqrt 3.0 / 4.0
        | Square s -> s * s
        | Rectangle(l, w) -> l * w

    interface IPrintable with // OO: implementatie van interface, maakt loose coupling mogelijk
        member this.Print () =
            match this with
            | Circle r -> printfn $"Circle with radius %f{r}"
            | EquilateralTriangle s -> printfn $"Equilateral Triangle of side %f{s}"
            | Square s -> printfn $"Square with side %f{s}"
            | Rectangle(l, w) -> printfn $"Rectangle with length %f{l} and width %f{w}"

### Vraag 3

Hieronder staat twee keer een simpel voorbeeldje uitgewerkt, eerst met exceptions, daarna met railway programming. Leg uit van welk voordeel van railway programming hier gebruik wordt gemaakt

In [ ]:
let rnd = System.Random()

// Met exceptions
exception EasyException of string
exception FatalException of string

let mayThrowA v = 
    let r = rnd.NextDouble ()
    if r < 0.5 then v + v
    elif r < 0.8 then raise (EasyException (v + " not a big problem")) 
    else raise (FatalException "uh oh...")

let mayThrowB v = 
    let r = rnd.NextDouble ()
    if r < 0.5 then v + v
    else raise (FatalException "uh oh...")

let doWorkOO v =
    try
        let v = mayThrowA v
        mayThrowB v
    with 
        | FatalException msg -> "fatal error"
        | EasyException msg ->  // Hier moet een geneste try-catch om recovery mogelijk te maken in OO met excepties
            try 
                mayThrowB msg
            with   
                | FatalException msg -> "fatal error"

printfn "OO: %s" (doWorkOO "hello")

// Met railways
let mayErrorA v = 
    let r = rnd.NextDouble ()
    if r < 0.5 then Ok (v + v)
    elif r < 0.8 then Error (EasyException (v + " not a big problem"))
    else Error (FatalException "uh oh...")

let mayErrorB v =
    let r = rnd.NextDouble ()
    if r < 0.5 then Ok (v + v)
    else Error (FatalException "uh oh...")

let connect f r = 
    match r with
        | Ok v -> f v
        | Error e -> Error e

let recover r = 
    match r with
        | Ok _-> r
        | Error (EasyException msg) -> Ok msg
        | Error _ -> r

let doWorkFP = mayErrorA >> recover >> connect mayErrorB // flow van het programma blijft altijd hetzelfde, recovery zit eenvoudig ingebouwd

printfn "FP: %s" (match doWorkFP "hello" with
    | Ok v -> v
    | Error (FatalException msg) -> "fatal error")

### Vraag 4

In [ ]:
// vraag a
let l = [Some (Some 1); Some (None); None; Some (Some 2); Some (None); Some (Some 3)]

// schrijf hier je uitwerking
let rec sumDoubleOption l =
    match l with
        | [] -> 0
        | None :: tail -> 1 + sumDoubleOption tail
        | Some None :: tail -> -1 + sumDoubleOption tail
        | Some (Some i) :: tail -> i + sumDoubleOption tail

printfn "%A" (sumDoubleOption l) // geeft 5

// vraag b
let rec sumPairs a b =
    match (a, b) with
        | ([], _) | (_, []) -> []
        | (Some ia :: ta, Some ib :: tb) -> ia + ib :: sumPairs ta tb
        | (_ :: ta, _ :: tb) -> sumPairs ta tb

let l1 = [Some 1; None; Some 3; Some 4; None; Some 5]
let l2 = [Some 1; None; None; Some 4; None; Some 5; None]

printfn "%A" (sumPairs l1 l2) // geeft [2; 8; 10]

// vraag c
let rec countResults results okCount errCount = 
    match results with
        | [] -> (okCount, errCount)
        | Ok _ :: tail -> countResults tail (okCount + 1) errCount
        | Error _ :: tail -> countResults tail okCount (errCount + 1)


printfn "%A" (countResults [Error "something went wrong"; Ok 5; Ok 3; Error "uh oh..."; Error "oopsie"] 0 0) 
// geeft: (2, 3)

### Vraag 5

Gegeven is een discriminated union `ContactInfo` waarmee een telefoonnummer en/of een emailadres kan worden vertegenwoordigd. Ook zijn er de functies `removePhone` en `setDefaultEmail` gegeven. `removePhone` verwijdert het telefoonnummer en `setDefaultEmail` voegt er een default emailadres aan toe. `setDefaultEmail` krijgt daarvoor een strategie (functie met signature `() -> string`) mee die nieuwe default adressen genereert.

Hier zie je voorbeelden van hoe de functies gebruikt kunnen worden:
```F#
removePhone (PhoneOnly "1234567890") // geeft Neither
removePhone (EmailAndPhone ("1234567890", "rma.vanderheiden@avans.nl")) // geeft EmailOnly "rma.vanderheiden@avans.nl"

let constantDefault () = "default@example.com"

setDefaultEmail constantDefault (EmailOnly "rma.vanderheiden@avans.nl") // geeft EmailOnly "rma.vanderheiden@avans.nl"
setDefaultEmail constantDefault (PhoneOnly "1234567890") // geeft PhoneAndEmail ("1234567890", "default@example.com")
```
1. Schrijf een functie `makeEmailFactory` die een default emailgenerator bouwt. `makeEmailFactory` geeft een functie terug met signature `() -> string` en iedere keer dat deze functie wordt aangeroepen wordt er een nieuw opvolgend defaultadres gegenereerd: "me0@example.com", "me1@example.com", "me2@example.com", et cetera. Je moet hierbij waarschijnlijk een mutable variabele gebruiken.
```F#
let emailFactory = makeEmailFactory ()

emailFactory () // geeft "me0@example.com"
emailFactory () // geeft "me1@example.com"
emailFactory () // geeft "me2@example.com"
```
2. Als vraag 3.1 niet helemaal correct werkt is de code voor deze vraag nog steeds goed te schrijven, maar de voorbeeldresultaten komen dan waarschijnlijk niet overeen. Schrijf een functie `getEmail` met signature `ContactInfo -> ContactInfo` die altijd een EmailOnly teruggeeft. Het emailadres dat erin zit moet het specifieke adres zijn als dat bestaat en anders gevuld zijn met een default adres. Schrijf deze functie op basis van `setDefaultEmail` en `removePhone` met functiecompositie. Gebruik hierbij partial application en de functie `makeEmailFactory`. (Dit gaat dus allemaal in 1 of 2 regels!)
```F#
getEmail Neither // EmailOnly "me0@example.com"
getEmail (PhoneOnly "1234567890") // EmailOnly "me1@example.com"
getEmail (EmailOnly "dj.koeze@avans.nl") // EmailOnly "dj.koeze@avans.nl"
getEmail (PhoneAndEmail ("1234567890", "dj.koeze@avans.nl")) // EmailOnly "dj.koeze@avans.nl"
```

In [ ]:
type ContactInfo =
    | Neither
    | PhoneOnly of string
    | EmailOnly of string
    | PhoneAndEmail of string * string

let removePhone info =
    match info with
        | Neither -> Neither // niets om te verwijderen
        | PhoneOnly _ -> Neither // niets blijft over met het telefoonnummer verwijderd
        | EmailOnly _ -> info // alleen een email, hoeft niets aan te veranderen
        | PhoneAndEmail (_, email) -> EmailOnly email // alleen de email blijft over


let setDefaultEmail makeDefaultEmail info =
    match info with
        | Neither -> EmailOnly (makeDefaultEmail ()) // zet alleen een defaultemail
        | PhoneOnly phone -> PhoneAndEmail (phone, (makeDefaultEmail ())) // voeg een defaultemail toe
        | EmailOnly _ -> info // hoeft niets te veranderen
        | PhoneAndEmail _ -> info // hoeft niets te veranderen


// vraag 1
let makeEmailFactory () =
    let mutable counter = 0
    fun () -> 
        let email = $"me{counter}@example.com"
        counter <- counter + 1
        email

// vraag 2
let emailFactory = makeEmailFactory ()
let getEmail = setDefaultEmail emailFactory >> removePhone

printfn "%O" (getEmail Neither) // EmailOnly "me0@example.com"
printfn "%O" (getEmail (PhoneOnly "1234567890")) // EmailOnly "me1@example.com"
printfn "%O" (getEmail (EmailOnly "dj.koeze@avans.nl")) // EmailOnly "dj.koeze@avans.nl"
printfn "%O" (getEmail (PhoneAndEmail ("1234567890", "dj.koeze@avans.nl"))) // EmailOnly "dj.koeze@avans.nl"

### Vraag 6

We hebben een operator nodig die de som bijhoudt van alle waarden die tot nu toe voorbij zijn gekomen. Deze operator werkt alleen op integers en hoeft dus niet generiek te zijn. De operator kan een mutable variabele gebruiken. Iedere keer wanneer er een nieuwe integer door de producer gemaakt wordt en bij de operator komt wordt de som bijgewerkt en de som verder doorgestuurd. Geef de operator een goede naam en werk de code uit.

Als volgende hebben we een operator nodig die waarden die hij binnenkrijgt herhaalt. Het aantal keer dat een waarde herhaald wordt moet meegegeven kunnen worden in een parameter (gebruik een goede volgorde zodat deze als eerste kan worden meegegeven met partial application). Deze operator heeft geen mutable variabelen nodig. 

In [ ]:
let duplicate amount previous =
    fun next ->
        previous (fun data -> 
            for i in 1..amount do next data
        )

let sum previous =
    let mutable total = 0
    fun next ->
        previous (fun data ->
            total <- total + data
            next total
        )

let producer observer = for i in 1..6 do observer i

let tripled = duplicate 3 producer
let summed = sum tripled

producer (printfn "you see me once: %i")
// you see me once: 1
// you see me once: 2
// you see me once: 3
// you see me once: 4
// you see me once: 5
// you see me once: 6

tripled (printfn "you see me three times: %i") 
// you see me three times: 1
// you see me three times: 1
// you see me three times: 1
// you see me three times: 2
// you see me three times: 2
// you see me three times: 2
// you see me three times: 3
// you see me three times: 3
// you see me three times: 3
// you see me three times: 4
// you see me three times: 4
// you see me three times: 4
// you see me three times: 5
// you see me three times: 5
// you see me three times: 5
// you see me three times: 6
// you see me three times: 6
// you see me three times: 6

summed (printfn "sum up to now: %i")
// sum up to now: 1
// sum up to now: 2
// sum up to now: 3
// sum up to now: 5
// sum up to now: 7
// sum up to now: 9
// sum up to now: 12
// sum up to now: 15
// sum up to now: 18
// sum up to now: 22
// sum up to now: 26
// sum up to now: 30
// sum up to now: 35
// sum up to now: 40
// sum up to now: 45
// sum up to now: 51
// sum up to now: 57
// sum up to now: 63

// geschreven als een pipeline:
(producer |> duplicate 3 |> sum) (printfn "sum from pipeline %i")